## This Notebook - Goals - FOR EDINA

**What?:**
- Standard classification method example/tutorial

**Who?:**
- Researchers in ML
- Students in computer science
- Teachers in ML/STEM

**Why?:**
- Demonstrate capability/simplicity of core scipy stack. 
- Demonstrate common ML concept known to learners and used by researchers.

**Noteable features to exploit:**
- use of pre-installed libraries: <code>numpy</code>, <code>scikit-learn</code>, <code>matplotlib</code>

**How?:**
- clear to understand - minimise assumed knowledge
- clear visualisations - concise explanations
- recognisable/familiar - use standard methods
- Effective use of core libraries

<hr>

# Classification - K nearest neighbours

K nearest neighbours is a simple and effective way to deal with classification problems. This method classifies each sample based on the class of the points that are closest to it.

This is a supervised learning method, meaning that data used contains information on some feature that the model should predict.

This notebook shows the process of classifying handwritten digits. 

<hr>

### Import libraries

On Noteable, all the libaries required for this notebook are pre-installed, so they simply need to be imported:

In [ ]:
import numpy as np

import sklearn.datasets as ds
import sklearn.model_selection as ms 

from sklearn import decomposition
from sklearn import neighbors
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline

<hr>

# Data - Handwritten Digits

In terms of data, [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) has a loading function for some data regarding hand written digits.

In [ ]:
# get the digits data from scikit into the notebook
digits = ds.load_digits()

The cell above loads the data as a [bunch object](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html), meaning that the data (in this case images of handwritten digits) and the target (the number that is written) can be split by accessing the attributes of the bunch object:

In [ ]:
# store data and targets seperately
X = digits.data
y = digits.target

print("The data is of the shape", X.shape)
print("The target data is of the shape", y.shape)

The individual samples in the <code>X</code> array each represent an image. In this representation, 64 numbers are used to represent a greyscale value on an 8\*8 square. The images can be examined by using pyplot's [matshow](https://matplotlib.org/3.3.0/api/_as_gen/matplotlib.pyplot.matshow.html) function.

The next cell displays the 17th sample in the dataset as an 8\*8 image.

In [ ]:
# create figure to display the 17th sample
fig = plt.matshow(digits.images[17], cmap=plt.cm.gray)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

Suppose instead of viewing the 17th sample, we want to see the average of samples corresponding to a certain value.

This can be done as follows (using 0 as an example):
- All samples where the target value is 0 are located
- The mean of these samples is taken
- The resulting 64 long array is reshaped to be 8\*8 (for display)
- The image is displayed

In [ ]:
# take samples with target=0
izeros = np.where(y == 0)
# take average across samples, reshape to visualise
zeros = np.mean(X[izeros], axis=0).reshape(8,8)

# display
fig = plt.matshow(zeros, cmap=plt.cm.gray)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

<hr>

# Fit and test the model

## Split the data

Now that you have an understanding of the data, the model can be fitted.

Fitting the model involves setting some of the data aside for testing, and allowing the model to "see" the target values corresponding to the training samples.

Once the model has been fitted to the training data, the model will be tested on some data it has not seen before. 

The next cell uses [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to shuffle all data, then set some data aside for testing later. 

For this example, $\frac{1}{4}$ of the data will be set aside for testing, and the model will be trained on the remaining training set.

As before, <code>X</code> corresponds to data samples, and <code>y</code> corresponds to labels.

In [ ]:
# split data to train and test sets
X_train, X_test, y_train, y_test = \
    ms.train_test_split(X, y, test_size=0.25, shuffle=True,
                       random_state=22)

The data can be examined - here you can see that 1347 samples have been put into the training set, and 450 have been set aside for testing.

In [ ]:
# print shape of data
print("training samples:", X_train.shape)  
print("testing samples :", X_test.shape)
print("training targets:", y_train.shape) 
print("testing targets :", y_test.shape)  

## Using PCA to visualise data

Before diving into classifying, it is useful to visualise the data.

Since each sample has 64 dimensions, some dimensionality reduction is needed in order to visualise the samples as points on a 2D map.

One of the easiest ways of visualising high dimensional data is by principal component analysis (PCA). This maps the 64 dimensional image data onto a lower dimension map (here we will map to 2D) so it can be easily viewed on a screen.

In this case, the 2 most important "components" are maintained.

In [ ]:
# create PCA model with 2 components
pca = decomposition.PCA(n_components=2)

The next step is to perform the PCA on the samples, and store the results.

In [ ]:
# transform training data to 2 principal components
X_pca = pca.fit_transform(X_train)

# transform test data to 2 principal components
T_pca = pca.transform(X_test)

In [ ]:
# check shape of result
print(X_pca.shape) 
print(T_pca.shape)

As you can see from the above cell, the <code>X_pca</code> and <code>T_pca</code> data is now represented by only 2 elements per sample. The number of samples has remained the same.

Now that there is a 2D representation of the data, it can be plotted on a regular scatter graph. Since the labels corresponding to each point are stored in the <code>y_train</code> variable, the plot can be colour coded by target value!

Different coloured dots have different target values.

In [ ]:
# choose the colours for each digit
cmap_digits = plt.cm.tab10

# plot training data with labels
plt.figure(figsize = (9,6))
plt.scatter(X_pca[:,0], X_pca[:,1], s=7, c=y_train,
            cmap=cmap_digits, alpha=0.7)
plt.title("Training data coloured by target value")
plt.colorbar();

## Create and fit the model

The scikit-learn library allows fitting of a k-NN model just as with PCA above.

First, create the classifier:

In [ ]:
# create model
knn = neighbors.KNeighborsClassifier()

The next step fits the k-NN model using the training data.

In [ ]:
# fit model to training data
knn.fit(X_train,y_train);

## Test model

Now use the data that was set aside earlier - this stage involves getting the model to "guess" the samples (this time without seeing their target values).

Once the model has predicted the sample's class, a score can be calculated by checking how many samples the model guessed correctly.

In [ ]:
# predict test data
preds = knn.predict(X_test)

# test model on test data
score = round(knn.score(X_test,y_test)*100, 2)
print("Score on test data: " + str(score) + "%")

98.44% is a really high score, one that would not likely be seen on real life applications of the method.

It can often be useful to visualise the results of your example. Below are plots showing:
- The labels that the model predicted for the test data
- The actual labels for the test data
- The data points that were incorrectly labelled

In this case, the predicted and actual plots are very similar, so these plots are not very informative. In other cases, this kind of visualisation may reveal patterns for you to explore further.

In [ ]:
# plot 3 axes
fig, axes = plt.subplots(2,2,figsize=(12,12))

# top left axis for predictions
axes[0,0].scatter(T_pca[:,0], T_pca[:,1], s=5, 
                  c=preds, cmap=cmap_digits)
axes[0,0].set_title("Predicted labels")

# top right axis for actual targets
axes[0,1].scatter(T_pca[:,0], T_pca[:,1], s=5, 
                  c=y_test, cmap=cmap_digits)
axes[0,1].set_title("Actual labels")

# bottom left axis coloured to show correct and incorrect
axes[1,0].scatter(T_pca[:,0], T_pca[:,1], s=5, 
                  c=(preds==y_test))
axes[1,0].set_title("Incorrect labels")

# bottom right axis not used
axes[1,1].set_axis_off()

So which samples did the model get wrong?

There were 7 samples that were misclassified. These can be displayed alongside their actual and predicted labels using the cell below:

In [ ]:
# find the misclassified samples
misclass = np.where(preds!=y_test)[0]

# display misclassified samples
r, c = 1, len(misclass)
fig, axes = plt.subplots(r,c,figsize=(10,5))

for i in range(c):
    ax = axes[i]
    ax.matshow(X_test[misclass[i]].reshape(8,8),cmap=plt.cm.gray)
    ax.set_axis_off()
    act = y_test[misclass[i]]
    pre = preds[misclass[i]]
    strng = "actual: {a:.0f} \npredicted: {p:.0f}".format(a=act, p=pre)
    ax.set_title(strng)

Additionally, a confusion matrix can be used to identify which samples are misclassified by the model. This can help you identify if their are samples that are commonly misidentified - for example you may identify that 8's are often mistook for 1's.

In [ ]:
# confusion matrix
conf = metrics.confusion_matrix(y_test,preds)

# figure
f, ax = plt.subplots(figsize=(9,5))
im = ax.imshow(conf, cmap=plt.cm.RdBu)

# set labels as ticks on axes
ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))
ax.set_xticklabels(list(range(0,10)))
ax.set_yticklabels(list(range(0,10)))
ax.set_ylim(9.5,-0.5)

# axes labels
ax.set_ylabel("actual value")
ax.set_xlabel("predicted value")
ax.set_title("Digit classification confusion matrix")

# display
plt.colorbar(im).set_label(label="number of classifications")